In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from generalization.randomization import available_corruptions

import matplotlib.pyplot as plt
import numpy as np
import os

def get_num_cpus():
    return len(os.sched_getaffinity(0))


def check(learn, dl, plt_title=None):
    y_pred, ys, losses = learn.get_preds(1, dl=dl, with_loss=True, reorder=True)

    print(y_pred[0].min(), y_pred[0].max(), y_pred[0].sum())
    print(y_pred[0].argmax(), ys[0], losses[0])
    plot_losses(losses, width=1, plt_title=plt_title)


def plot_losses(losses, width=1, plt_title=None):
    plt.bar(np.arange(len(losses)), losses, width=width)
    plt.title(plt_title)
    plt.show()


print("Available corruptions:\n", available_corruptions())

Available corruptions:
 ['gaussian_pixels', 'random_labels', 'random_pixels', 'partial_labels', 'shuffled_pixels']


## Configuration for all experiments

In [3]:
from generalization.randomization import build_cifar10
import torch


def build_experiments(corrupt_prob, corrupt_name=None, batch_size=128):
    corruptions = available_corruptions()

    experiments = dict()

    if corrupt_name is not None:
        corruptions = [corrupt_name]

    for corrupt_name in corruptions:
        if corrupt_name == "gaussian_pixels":
            continue
        train_set, test_set = build_cifar10(
            corruption_name=corrupt_name,
            corruption_prob=corrupt_prob,
            show_images=False,
            verbose=False,
        )

        train_loader = torch.utils.data.DataLoader(
            train_set,
            batch_size=batch_size,
            shuffle=True,
            num_workers=get_num_cpus(),
            pin_memory=True,
        )

        val_set, test_set = torch.utils.data.random_split(
            test_set, [len(test_set) // 2, len(test_set) - len(test_set) // 2]
        )

        val_loader = torch.utils.data.DataLoader(
            val_set,
            batch_size=batch_size * 2,
            shuffle=False,
            num_workers=get_num_cpus(),
            pin_memory=True,
        )

        test_loader = torch.utils.data.DataLoader(
            test_set,
            batch_size=batch_size * 2,
            shuffle=False,
            num_workers=get_num_cpus(),
            pin_memory=True,
        )

        experiments[corrupt_name] = {
            "train_set": train_set,
            "val_set": val_set,
            "test_set": test_set,
            "train_loader": train_loader,
            "val_loader": val_loader,
            "test_loader": test_loader,
        }
    return experiments

## All Corruptions

In [4]:
import lightning as L
import torchmetrics
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.loggers import TensorBoardLogger

from torch import nn
from torch.nn import functional as F


class LitModel(L.LightningModule):
    def __init__(self, net: nn.Module, lr: float = 1e-3, n_classes=10):
        super().__init__()
        self.net = net
        self.lr = lr
        self.n_classes = n_classes
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)
        self.valid_acc = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)
        self.valid_top5_acc = torchmetrics.Accuracy(
            task="multiclass", num_classes=n_classes, top_k=5
        )

        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)
        self.test_top5_acc = torchmetrics.Accuracy(
            task="multiclass", num_classes=n_classes, top_k=5
        )

        self.early_stop_counter = 0
        self.patience = 5
        self.best_valid_acc = 0

    def forward(self, x):
        out = self.net(x)
        return out

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        self.log("train/loss", loss, on_epoch=True, on_step=False, prog_bar=True)
        self.step_metrics(logits=logits, y=y, mode="train")
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        self.log("valid/loss", loss, on_epoch=True, on_step=False, prog_bar=True)
        self.step_metrics(logits=logits, y=y, mode="val")
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        self.log("test/loss", loss, on_epoch=True, on_step=False, prog_bar=True)
        self.step_metrics(logits=logits, y=y, mode="val")
        return loss

    def on_validation_end(self) -> None:
        current_valid_acc = self.valid_acc.compute()
        if current_valid_acc > self.best_valid_acc:
            self.best_valid_acc = current_valid_acc
            self.early_stop_counter = 0
        else:
            self.early_stop_counter += 1
            if self.early_stop_counter > self.patience:
                self.early_stop_counter = 0
                # self.trainer.should_stop = True
                log_fn = (
                    self.logger.experiment.add_scalar
                    if isinstance(self.logger, TensorBoardLogger)
                    else self.logger.experiment.log
                )

                log_fn({"early_stop": self.current_epoch})

    def step_metrics(self, logits, y, mode):
        if mode == "train":
            self.train_acc.update(logits, y)
            self.log(
                "train/acc", self.train_acc, on_step=False, on_epoch=True, prog_bar=True
            )

        elif mode == "val":
            self.valid_acc.update(logits, y)
            self.valid_top5_acc.update(logits, y)

            self.log(
                "valid/acc",
                self.valid_acc.compute(),
                on_step=False,
                on_epoch=True,
                prog_bar=True,
            )
            self.log(
                "valid/top5_acc",
                self.valid_top5_acc.compute(),
                on_step=False,
                on_epoch=True,
                prog_bar=True,
            )
        elif mode == "test":
            self.test_acc.update(logits, y)
            self.test_top5_acc.update(logits, y)
            self.log("test/acc", self.test_acc.compute(), on_step=False, on_epoch=True)
            self.log(
                "test/top5_acc",
                self.test_top5_acc.compute(),
                on_step=False,
                on_epoch=True,
            )

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.net.parameters(), lr=self.lr, momentum=0.9)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.99)
        return [optimizer], [scheduler]

In [5]:
import time
import wandb
from generalization.models import get_cifar_models

torch.set_float32_matmul_precision("medium")

SEED = 88
BATCH_SIZE = 256
LEARNING_RATE = 0.01
PRINT_EVERY = 3
EPOCHS = 60
MODEL_NAME = "inception"
CORRUPT_NAME = "random_labels"

L.seed_everything(SEED)
# experiments = build_experiments(corrupt_prob=0.1, corrupt_name=CORRUPT_NAME, batch_size=BATCH_SIZE)

# models = get_cifar_models(lib="torch")
# print(models.keys())
# net = models.get(MODEL_NAME)

# logger = None
# logger = WandbLogger(
#     name=f"{MODEL_NAME}",
#     project="generalization",
#     log_model=True,
#     save_dir="logs",
#     version="2",
#     id=f"{MODEL_NAME}-{CORRUPT_NAME}",
#     group=f"{CORRUPT_NAME}",
#     tags=[MODEL_NAME, CORRUPT_NAME],
# )

# logger = TensorBoardLogger(
#     save_dir="logs",
#     name=f"{MODEL_NAME}",
#     version=f"{CORRUPT_NAME}",
# )

Global seed set to 88


88

In [6]:
# trainer = L.Trainer(
#     max_epochs=60, 
#     logger=logger , 
#     default_root_dir="logs",
# )
# pl_model = LitModel(net, lr=LEARNING_RATE, n_classes=10)
# start_time = time.time()
# trainer.fit(
#     pl_model,
#     experiments[CORRUPT_NAME]["train_loader"],
#     experiments[CORRUPT_NAME]["val_loader"],
# )
# print(f"Training took {time.time() - start_time:.2f} seconds")

In [7]:
for corrupt_name in available_corruptions():
    if corrupt_name == "gaussian_pixels":
        continue
    print(f"Corruption: {corrupt_name}")
    experiments = build_experiments(
        corrupt_prob=0.1, corrupt_name=corrupt_name, batch_size=BATCH_SIZE
    )
    corrup_probs = (
        [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.9]
        if not "random" in corrupt_name
        else [1]
    )

    for corrupt_prob in corrup_probs:
        print(f"Corruption prob: {corrupt_prob}")
        for model_name, net in get_cifar_models(lib="torch").items():
            print(f"Model: {model_name}")

            MODEL_NAME = model_name
            CORRUPT_NAME = corrupt_name

            logger = WandbLogger(
                name=f"{MODEL_NAME}-{corrupt_prob}",
                project="generalization",
                log_model=True,
                save_dir="logs",
                version="3",
                id=f"{MODEL_NAME}-{CORRUPT_NAME}-{corrupt_prob}",
                group=f"{CORRUPT_NAME}",
                tags=[MODEL_NAME, CORRUPT_NAME],
            )

            trainer = L.Trainer(
                max_epochs=60,
                logger=logger,
                default_root_dir="logs",
            )
            pl_model = LitModel(net, lr=LEARNING_RATE, n_classes=10)
            start_time = time.time()
            trainer.fit(
                pl_model,
                experiments[CORRUPT_NAME]["train_loader"],
                experiments[CORRUPT_NAME]["val_loader"],
            )
            print(f"Training took {time.time() - start_time:.2f} seconds")

            trainer.test(pl_model, experiments[CORRUPT_NAME]["test_loader"])

            # assure that logger process has exited
            trainer.logger.experiment.finish()

Corruption: random_labels
Files already downloaded and verified


/home/step/Code/projects/ids-generalization/notebooks/generalization/randomization/dataset.py:203: UserWarning: corruption_prob is ignored when corruption_name is 'random_*'
  warnings.warn(


Files already downloaded and verified
Corruption prob: 1


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Model: alexnet


...

Training took 426.53 seconds
Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 129.23it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/loss         │     35.67580795288086     │
│         valid/acc         │   0.024206291884183884    │
│      valid/top5_acc       │    0.2917298376560211     │
└───────────────────────────┴───────────────────────────┘

/home/step/mambaforge/envs/generalization/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/step/mambaforge/envs/generalization/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory logs/generalization/3/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params
------------------------------------------------------
0 | net            | InceptionSmall     | 8.0 M 
1 | train_acc      | Multic

Model: inception
Epoch 10:  18%|█▊        | 35/196 [00:02<00:09, 17.07it/s, v_num=3, valid/loss=18.90, valid/acc=0.0211, valid/top5_acc=0.311, train/loss=0.110, train/acc=0.962]

Epoch 10:  38%|███▊      | 74/196 [00:03<00:05, 23.27it/s, v_num=3, valid/loss=18.90, valid/acc=0.0211, valid/top5_acc=0.311, train/loss=0.110, train/acc=0.962]

In [ ]:
trainer.logger.experiment.log?

In [ ]:
from torchvision import transforms
from generalization.randomization.utils import CIFAR10_NORMALIZE_MEAN, CIFAR10_NORMALIZE_STD


idx = np.random.randint(len(experiments[CORRUPT_NAME]["train_set"]))

unnormalize = transforms.functional.normalize(
    experiments[CORRUPT_NAME]["train_set"][idx][0],
    mean=[-m / s for m, s in zip(CIFAR10_NORMALIZE_MEAN, CIFAR10_NORMALIZE_STD)],
    std=[1 / s for s in CIFAR10_NORMALIZE_STD],
)

label = experiments[CORRUPT_NAME]["train_set"][idx][1]
class_name = experiments[CORRUPT_NAME]["train_set"].classes[label]


f, ax = plt.subplots(1, 1, figsize=(2, 2))
ax.imshow(unnormalize.permute(1, 2, 0))
ax.set_title(f"Corrupted label: {class_name}")
ax.axis("off")
plt.show()

In [ ]:
# !tensorboard --logdir logs

In [ ]:
# import wandb
# 
# for model in models.keys():
#     print("Model:", model)

#     for corruption, exp in experiments.items():
#         wandb.finish()
#         print("Corruption:", corruption)

#         net = models.get(model)
#         logger = WandbLogger(
#             name=f"{model}",
#             project="generalization",
#             log_model=True,
#             save_dir="logs",
#             group=f"{corruption}"
#         )
#         trainer = L.Trainer(max_epochs=10, logger=logger, default_root_dir="logs")
#         pl_model = LitModel(net)
#         start_time = time.time()
#         trainer.fit(
#             pl_model,
#             experiments[corruption]["train_loader"],
#             experiments[corruption]["test_loader"],
#         )
#         print(
#             f"== Total time: {time.time() - start_time:.3f} s, {(time.time() - start_time)/ 60:.3f} min"
#         )
#         break